# Scrape Venmo

## Imports

In [1]:
import glob
import json
import time
import re
import os

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup
from IPython.display import clear_output

## Set up the scraper

In [2]:
driver = webdriver.Firefox()

In [3]:
base_url = 'https://venmo.com/api/v5/'

In [4]:
venmo_user = 'frtennis1@gmail.com'
# os.environ['venmo_pwd'] = 

In [5]:
def log_in():
    driver.get('https://venmo.com')
    sign_in_btn = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.LINK_TEXT, 'Sign In')))
    sign_in_btn.click()
    email_box = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.NAME, 'phoneEmailUsername')))
    email_box.send_keys(venmo_user)
    pwd_box = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.NAME, 'password')))
    pwd_box.send_keys(os.environ['venmo_pwd'])
    pwd_box.submit()

In [6]:
def query_api(url):
    driver.get(url)
    raw_data_btn = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, 'Raw Data')))
    raw_data_btn.click()
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'data'))
    )
    return element.text

In [7]:
def get_and_save_feed_api_call(url, u_id, i):
    s = query_api(url)
    with open('data/raw/%s-%03i.json' % (u_id, i), 'w') as f:
        f.write(s)
    data = json.loads(s)
    if 'error' in data:
        raise ValueError('Error in API call')
    return data

In [8]:
def pull_down_profile(u_id, verbose=True):
    i = 0
    first_url = base_url + '/users/%s/feed' % u_id
    data = get_and_save_feed_api_call(first_url, u_id, i)
    
    cont = True
    
    while ('paging' in data 
           and 'next' in data['paging'] 
           and data['paging']['next'] 
           and cont):
        if verbose:
            print(data['paging']['next'])
        i += 1
        data = get_and_save_feed_api_call(data['paging']['next'], u_id, i)
        
        cont = True

## Run the scraper

Get the list of IDs to crawl from the database creator notebook.

In [9]:
%store -r to_crawl_ids

no stored variable to_crawl_ids


In [10]:
log_in()

KeyError: 'venmo_pwd'

In [409]:
# run this cell only for debugging purposes if the loop terminated early
id_num

11620419

In [443]:
def del_profile(id_num):
    files = glob.glob('data/raw/%i-*' % id_num)
    for f in files:
        os.remove(f)

In [442]:
# delete an incomplete profile so that it can get pulled later
# del_profile(123)

In [448]:
failed = True
while failed:
    try:
        failed = False
        for id_num in tqdm(to_crawl_ids):
            if glob.glob('data/raw/%i-*' % id_num):
                continue
            pull_down_profile(id_num, verbose=False)

    except TimeoutException:
        failed = True
        clear_output()
        print('Timed out. Sleeping for 20 seconds...')
        del_profile(id_num)
        time.sleep(20)
    except ValueError:
        failed = True
        clear_output()
        print('ValueError, trying to log back in...')
        del_profile(id_num)
        log_in()
        time.sleep(10)

ValueError, trying to log back in...
176/|/ 11%|| 176/1590 [00:13<01:49, 12.96it/s]

TimeoutException: Message: Timeout loading page after 300000ms
